In [0]:
%sql
-- カタログとスキーマ（存在しない場合のみ作成）
CREATE CATALOG IF NOT EXISTS hiroshi;
CREATE SCHEMA  IF NOT EXISTS hiroshi.ai_demo_hub;

CREATE TABLE IF NOT EXISTS hiroshi.ai_demo_hub.demos (
  -- 自動採番 PK (IDENTITY)
  demo_id BIGINT GENERATED BY DEFAULT AS IDENTITY
          COMMENT '自動採番 ID',

  /* ===== 基本情報 ===== */
  title          STRING  NOT NULL COMMENT 'デモのタイトル',
  summary        STRING           COMMENT 'カード表示用の要約',
  description_md STRING           COMMENT '詳細説明 (Markdown)',

  /* ===== 投稿者・日時 ===== */
  creator_emp_id STRING  COMMENT 'デモの作成者の社員ID',
  owner_emp_id   STRING  NOT NULL COMMENT '代表投稿者の社員ID',
  created_at     TIMESTAMP NOT NULL DEFAULT current_timestamp()
                 COMMENT '登録日時',
  updated_at     TIMESTAMP         COMMENT '最終編集日時',

  /* ===== ステータス ===== */
  status         STRING  NOT NULL COMMENT 'draft / in_review / published / archived',

  /* ===== リンク ===== */
  demo_url       STRING  NOT NULL COMMENT '実デモ URL',
  repo_url       STRING           COMMENT 'ソースコードリポジトリ',

  /* ===== 技術スタック ===== */
  products       ARRAY<STRING> NOT NULL COMMENT '利用製品名の配列',

  /* ===== ガバナンス ===== */
  confidentiality STRING COMMENT 'public / internal',

  remarks        STRING COMMENT '備考',
  all_info_md        STRING COMMENT 'すべての情報をマークダウン形式で記述',

  /* ===== 制約 ===== */
  CONSTRAINT demos_pk PRIMARY KEY (demo_id)
)
USING DELTA
PARTITIONED BY (status)
TBLPROPERTIES (
  -- 🌟 ここで列デフォルト機能を有効化
  delta.feature.allowColumnDefaults = 'supported',

  -- 🌟 IDENTITY 列もテーブル機能なので明示しておくと安全
  delta.feature.identityColumns     = 'supported',

  -- 既存プロパティ
  delta.columnMapping.mode          = 'name',           -- 高度機能利用時の推奨設定 [oai_citation:4‡Microsoft Learn](https://learn.microsoft.com/en-us/azure/databricks/sql/language-manual/sql-ref-syntax-ddl-create-table-using?utm_source=chatgpt.com)
  delta.enableChangeDataFeed        = true,
  delta.logRetentionDuration        = 'interval 30 days'
)
COMMENT '社内 AI デモのメタデータカタログ';

In [0]:
%sql
ALTER TABLE hiroshi.ai_demo_hub.demos
  ADD CONSTRAINT status_chk CHECK (status IN ('draft','in_review','published','archived'));

ALTER TABLE hiroshi.ai_demo_hub.demos
  ADD CONSTRAINT conf_chk CHECK (confidentiality IN ('public','internal'));